<a href="https://colab.research.google.com/github/shyousefi/Emotion-Detection/blob/main/GoEmotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import the libraries

In [ ]:
!pip install torch
!pip install emoji
!pip install transformers


In [ ]:
import torch
import pandas as pd
import re
import string
import emoji
from bs4 import BeautifulSoup

# Load the data

In [ ]:
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv

--2023-09-22 20:00:51--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.207, 142.250.31.207, 142.251.163.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.63.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14174600 (14M) [application/octet-stream]
Saving to: ‘data/full_dataset/goemotions_1.csv’

goemotions_1.csv    100%[===================>]  13.52M  89.6MB/s    in 0.2s    

2023-09-22 20:00:51 (89.6 MB/s) - ‘data/full_dataset/goemotions_1.csv’ saved [14174600/14174600]

--2023-09-22 20:00:51--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.207, 142.250.31.207, 142.251.163.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.63.207|:443... connected.
HTTP request sent, awaiting res

In [ ]:
file_paths = ['/content/data/full_dataset/goemotions_1.csv', '/content/data/full_dataset/goemotions_2.csv', '/content/data/full_dataset/goemotions_2.csv']

merged_df = pd.concat(
    [
        pd.read_csv(file_path)
        for file_path in file_paths
    ],
    ignore_index=True,
)

print ('shape merged_df:',merged_df.shape)
print('unique text merged_df:', len(merged_df['text'].unique()))
print('unique rater_id merged_df:', len(merged_df['rater_id'].unique()))

shape merged_df: (210000, 37)
unique text merged_df: 55842
unique rater_id merged_df: 82


# Preprocessing

In [ ]:
# keep the necessary data
final_df = merged_df[['text', 'rater_id', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']]

print ('shape final_df:',final_df.shape)
print('unique text final_df:', len(final_df['text'].unique()))
print('unique rater_id final_df:', len(final_df['rater_id'].unique()))

shape final_df: (210000, 9)
unique text final_df: 55842
unique rater_id final_df: 82


In [ ]:
# find zero_rows
zero_rows = final_df.index[final_df[['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']].eq(0).all(1)]
print (len(zero_rows))

174983


In [ ]:
# delete zero_rows for avoiding NAN values
final_df = final_df.drop(zero_rows)

print ('shape final_df:',final_df.shape)
print('unique text final_df:', len(final_df['text'].unique()))
print('unique rater_id final_df:', len(final_df['rater_id'].unique()))

shape final_df: (35017, 9)
unique text final_df: 16461
unique rater_id final_df: 81


In [ ]:
# small data set for test: due to limited resources
shuffled_df = final_df.sample(frac=1)
data = shuffled_df.head(1000)

print ('shape data:',data.shape)
print('unique text data:', len(data['text'].unique()))
print('unique rater_id data:', len(data['rater_id'].unique()))

shape data: (1000, 9)
unique text data: 973
unique rater_id data: 69


In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am",
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not",
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will",
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have",
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is",
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-',
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does',
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum',
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend',
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [ ]:
def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    # Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text

def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''
    for p in mapping:
        text = text.replace(p, mapping[p])

    for p in punct:
        text = text.replace(p, f' {p} ')

    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])

    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''
    #Removes awkward spaces
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(df, column_name):
    '''Cleaning and parsing the text in the specified column of the dataframe.'''
    df[column_name] = df[column_name].apply(clean_text)
    df[column_name] = df[column_name].apply(clean_contractions, mapping=contraction_mapping)
    df[column_name] = df[column_name].apply(clean_special_chars, punct=punct, mapping=punct_mapping)
    df[column_name] = df[column_name].apply(correct_spelling, dic=mispell_dict)
    df[column_name] = df[column_name].apply(remove_space)
    return df

In [ ]:
def remove_emojis(df,col):
  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

  df["text"] = df[col].str.replace(emoji_pattern, '',regex=True)

  return df

In [ ]:
df_clean = text_preprocessing_pipeline(data, 'text')
df_clean = remove_emojis(data, 'text')

<ipython-input-11-af9cabbc7eb0>:15: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'lxml').get_text()
<ipython-input-11-af9cabbc7eb0>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(clean_text)
<ipython-input-11-af9cabbc7eb0>:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(clean_contracti

# Method 1

In [ ]:
df = df_clean

In [ ]:
print (df.shape)

(1000, 9)


In [ ]:
# imprt the pre_train model
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("justin871030/bert-base-uncased-goemotions-ekman-finetuned")
model = AutoModelForSequenceClassification.from_pretrained("justin871030/bert-base-uncased-goemotions-ekman-finetuned")

In [ ]:
# culculate the emotions
def classify_emotions(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = logits.softmax(dim=1)
    return probabilities.detach().numpy()[0]

In [ ]:
emotions = df['text'].apply(classify_emotions)

In [ ]:
emotions

97547     [0.14077617, 0.13840866, 0.11913217, 0.1560493...
83917     [0.14429829, 0.14030764, 0.11981348, 0.1494001...
133561    [0.14085549, 0.13824971, 0.1194612, 0.15786436...
5922      [0.14504589, 0.13969754, 0.120175384, 0.151158...
57432     [0.14324196, 0.13972758, 0.12240541, 0.1512579...
                                ...                        
196285    [0.14240594, 0.13976885, 0.12037624, 0.1512898...
43296     [0.146777, 0.14001432, 0.12008378, 0.15014479,...
63273     [0.14422987, 0.140214, 0.12009595, 0.14969207,...
147213    [0.14058098, 0.13812202, 0.11975519, 0.1584600...
130295    [0.1465715, 0.13964991, 0.12021564, 0.15075897...
Name: text, Length: 1000, dtype: object

In [ ]:
emotions_df = pd.DataFrame(emotions.tolist(), columns=['anger*', 'disgust*', 'fear*', 'joy*', 'sadness*', 'surprise*', 'neutral*'])
# result_df = pd.concat([df, emotions_df], axis=1) # incliding NAN values
result_df = pd.concat([df.reset_index(drop=True), emotions_df.reset_index(drop=True)], axis=1) # delete the NAN values
print(result_df.shape)

(1000, 16)


In [ ]:
result_df

,text,rater_id,anger,disgust,fear,joy,sadness,surprise,neutral,anger*,disgust*,fear*,joy*,sadness*,surprise*,neutral*
0,thank you i am surprised with the amount of fe...,49,0,0,0,0,0,1,0,0.140776,0.138409,0.119132,0.156049,0.156124,0.133699,0.155811
1,what is with people microwaving cheese on top ...,38,1,0,0,0,0,0,0,0.144298,0.140308,0.119813,0.149400,0.161216,0.131323,0.153641
2,oh the joys of marine corps boot camp,42,0,0,0,1,0,0,0,0.140855,0.138250,0.119461,0.157864,0.156231,0.134087,0.153251
3,he is also a chaser and is convinced that shou...,52,0,1,0,0,0,0,0,0.145046,0.139698,0.120175,0.151158,0.160707,0.131634,0.151582
4,probably government contracts are already terr...,24,0,0,1,0,0,0,0,0.143242,0.139728,0.122405,0.151258,0.158433,0.133068,0.151866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,wait is this fucking loss,27,1,0,0,0,0,0,0,0.142406,0.139769,0.120376,0.151290,0.157896,0.135200,0.153064
996,you re an asshole,4,1,0,0,0,0,0,0,0.146777,0.140014,0.120084,0.150145,0.160590,0.131382,0.151008
997,so a kid,61,0,0,0,0,0,1,0,0.144230,0.140214,0.120096,0.149692,0.162126,0.131296,0.152345
998,got a nice hairdoodoo,2,0,0,0,1,0,0,0,0.140581,0.138122,0.119755,0.158460,0.155704,0.134524,0.152853


In [ ]:
max_rates = df.groupby(['text', 'rater_id']).max().reset_index()

In [ ]:
majority_vote = max_rates.groupby('text').sum().apply(lambda x: x > len(x)/2)

In [ ]:
result_df = result_df[['anger*', 'disgust*', 'fear*', 'joy*', 'sadness*', 'surprise*']]
majority_vote = majority_vote [['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']]
majority_vote.columns = ['anger*', 'disgust*', 'fear*', 'joy*', 'sadness*', 'surprise*']
print('result_df.columns', len(result_df.columns))
print('majority_vote', len(majority_vote.columns))

result_df.columns 6
majority_vote 6


In [ ]:
result_df

,anger*,disgust*,fear*,joy*,sadness*,surprise*
0,0.140776,0.138409,0.119132,0.156049,0.156124,0.133699
1,0.144298,0.140308,0.119813,0.149400,0.161216,0.131323
2,0.140855,0.138250,0.119461,0.157864,0.156231,0.134087
3,0.145046,0.139698,0.120175,0.151158,0.160707,0.131634
4,0.143242,0.139728,0.122405,0.151258,0.158433,0.133068
...,...,...,...,...,...,...
995,0.142406,0.139769,0.120376,0.151290,0.157896,0.135200
996,0.146777,0.140014,0.120084,0.150145,0.160590,0.131382
997,0.144230,0.140214,0.120096,0.149692,0.162126,0.131296
998,0.140581,0.138122,0.119755,0.158460,0.155704,0.134524


In [ ]:
# replace the decimal values with integers
def replace_max(row):
  max_value = row.max()
  return [1 if x == max_value else 0 for x in row]

# apply the function to each row of the DataFrame
result_df = result_df.apply(replace_max, axis=1, result_type='expand')

print(result_df)

     0  1  2  3  4  5
0    0  0  0  0  1  0
1    0  0  0  0  1  0
2    0  0  0  1  0  0
3    0  0  0  0  1  0
4    0  0  0  0  1  0
..  .. .. .. .. .. ..
995  0  0  0  0  1  0
996  0  0  0  0  1  0
997  0  0  0  0  1  0
998  0  0  0  1  0  0
999  0  0  0  0  1  0

[1000 rows x 6 columns]


In [ ]:
result_df.columns = ['anger*', 'disgust*', 'fear*', 'joy*', 'sadness*', 'surprise*']
result_df

,anger*,disgust*,fear*,joy*,sadness*,surprise*
0,0,0,0,0,1,0
1,0,0,0,0,1,0
2,0,0,0,1,0,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
...,...,...,...,...,...,...
995,0,0,0,0,1,0
996,0,0,0,0,1,0
997,0,0,0,0,1,0
998,0,0,0,1,0,0


In [ ]:
majority_vote = majority_vote.astype(int)
result_df = result_df.astype(int)

In [ ]:
accuracy = (result_df[['anger*', 'disgust*', 'fear*', 'joy*', 'sadness*', 'surprise*']].index.difference == majority_vote.index).mean()